In [1]:
import time
import logging
from binance.lib.utils import config_logging
from binance.websocket.futures.websocket_client import FuturesWebsocketClient as Client
import pandas as pd
import matplotlib
import numpy as np
config_logging(logging, logging.DEBUG)

In [2]:
def message_handler(message, df):
    try:
      row = message["k"]
      row["event_time"] = message["E"]
      if len(df) < 52:      
        df.append(row)
      else:
        df.pop(0)
        df.append(row)
    except:
      print(message)

    # print(dfs)

kl_list = []

my_client = Client()
my_client.start()


my_client.continuous_kline(
    pair="btcusdt",
    id=1,
    contractType="perpetual", 
    interval='1d',
    callback=lambda m: message_handler(m, kl_list),
)

time.sleep(10)

logging.debug("closing ws connection")
my_client.stop()

INFO:root:Connection with URL: wss://fstream.binance.com/ws
INFO:root:Start to connect....
INFO:root:Server connected
INFO:root:Sending message to Server: b'{"method": "SUBSCRIBE", "params": ["btcusdt_perpetual@continuousKline_1d"], "id": 1}'


{'id': 1, 'result': None}


DEBUG:root:closing ws connection


In [3]:
def process_ws_klines(kl_list):

  df = pd.DataFrame(kl_list)
  df.drop(labels=["i", "f", "L", "B"], axis=1, inplace=True)
  
  reordered_labels = ['event_time', 't', 'o', 'h', 'l', 'c', 'v', 'T', 'q', 'n', 'V', 'Q']
  
  df = df.reindex(columns=reordered_labels)
  df.columns = ['event_time', 'open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'trades', 'taker_buy_volume', 'taker_buy_quote_asset_volume']
  df['event_time'] = pd.to_datetime(df['event_time'], unit='ms')
  df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
  df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
  df['open'] = pd.to_numeric(df['open'])
  df['high'] = pd.to_numeric(df['high'])
  df['low'] = pd.to_numeric(df['low'])
  df['close'] = pd.to_numeric(df['close'])
  df['volume'] = pd.to_numeric(df['volume'])
  df['quote_asset_volume'] = pd.to_numeric(df['quote_asset_volume'])
  df['trades'] = pd.to_numeric(df['trades'])
  df['taker_buy_volume'] = pd.to_numeric(df['taker_buy_volume'])
  df['taker_buy_quote_asset_volume'] = pd.to_numeric(df['taker_buy_quote_asset_volume'])

  return df



In [4]:
df = process_ws_klines(kl_list=kl_list)
#%%
df

,event_time,open_time,open,high,low,close,volume,close_time,quote_asset_volume,trades,taker_buy_volume,taker_buy_quote_asset_volume
0,2022-05-01 01:44:10.637,2022-05-01,37614.4,37866.6,37350.0,37836.7,29775.993,2022-05-01 23:59:59.999,1.120330e+09,328620,14372.470,5.410901e+08
1,2022-05-01 01:44:10.902,2022-05-01,37614.4,37866.6,37350.0,37841.7,29786.487,2022-05-01 23:59:59.999,1.120727e+09,328680,14382.892,5.414845e+08
2,2022-05-01 01:44:11.154,2022-05-01,37614.4,37866.6,37350.0,37845.0,29786.984,2022-05-01 23:59:59.999,1.120746e+09,328697,14383.265,5.414986e+08
3,2022-05-01 01:44:11.427,2022-05-01,37614.4,37866.6,37350.0,37846.9,29788.369,2022-05-01 23:59:59.999,1.120798e+09,328724,14384.292,5.415375e+08
4,2022-05-01 01:44:11.719,2022-05-01,37614.4,37866.6,37350.0,37847.1,29791.432,2022-05-01 23:59:59.999,1.120914e+09,328760,14385.144,5.415697e+08
5,2022-05-01 01:44:12.049,2022-05-01,37614.4,37866.6,37350.0,37841.2,29794.642,2022-05-01 23:59:59.999,1.121036e+09,328810,14385.202,5.415719e+08
6,2022-05-01 01:44:12.310,2022-05-01,37614.4,37866.6,37350.0,37845.8,29797.961,2022-05-01 23:59:59.999,1.121161e+09,328855,14388.029,5.416789e+08
7,2022-05-01 01:44:12.566,2022-05-01,37614.4,37866.6,37350.0,37845.3,29798.141,2022-05-01 23:59:59.999,1.121168e+09,328863,14388.208,5.416857e+08
8,2022-05-01 01:44:12.820,2022-05-01,37614.4,37866.6,37350.0,37845.5,29798.523,2022-05-01 23:59:59.999,1.121183e+09,328878,14388.420,5.416937e+08
9,2022-05-01 01:44:13.073,2022-05-01,37614.4,37866.6,37350.0,37847.1,29799.486,2022-05-01 23:59:59.999,1.121219e+09,328891,14389.114,5.417200e+08
